# SSM 계산 : 차량 페어, 롱타입
* Heatmap으로 나타내기
* 시공도로 나타내기

# Import

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

import math # arctangent; math.atan 사용 목적
import statistics

from SSM_base import *
from SSM_time_based import *
from SSM_deceleration_based import *
from SSM_energy_based import *

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

In [ ]:
help(points)

# Load Dataset

In [2]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [3]:
file_list = ['LV_long_trajectories-0750am-0805am.parquet', 'LV_long_trajectories-0805am-0820am.parquet', 'LV_long_trajectories-0820am-0835am.parquet']

In [ ]:
folder_name = '02_processed_long'

for file, i in zip(file_list[:1], [1, 2, 3][:1]):
    file_path = os.path.join(working_dir, folder_name, file)
    df = pq.read_pandas(file_path).to_pandas()
    
    df.rename({'LV_V_len' : 'LV_len', 'LV_V_wid' : 'LV_wid'}, axis = 1, inplace = True)
    
    globals()[f'df_{i}'] = df

In [ ]:
df_1.head(3)

# Calculation

## Settings
* 공통 상수 설정

In [ ]:
threshold_TTC = 1.5 # time-based SSM 임계값
threshold_DRAC = 3.4

tmsec = 0.1 # 각 row별 시간단위(초)
g = 9.81 # 중력가속도
f = 0.35 # 마찰계수. friction coeefficient
delta_t = 1 # Reaction Delay time. 운전자 반응시간
MADR = 8.45 # 차량의 최대 감속 능력. 날씨에 따라, 차종에 따라 다르다. Saccomanno et al.(2008) 참조
deceleration_rate = 0.3 * g # 차량 감속도. 3.3도 됨

## Time-based SSM
* Temporal Proximal Indicator

In [ ]:
# 꼭지점 4개 만들기
# 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.

tqdm.pandas()

warnings.filterwarnings('ignore')

for df, i in zip([df_1][:1], [1][:1]):
    
        
    print(f'FILE {i} :: Basics Started')
    df['new_local_x'], df['new_local_y'] = zip(*df.progress_apply(lambda x: NGSIM_center(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
    df['new_LV_local_x'], df['new_LV_local_y'] = zip(*df.progress_apply(lambda x: NGSIM_center(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))
    
    df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.progress_apply(lambda x: points(x.new_local_x, x.new_local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
    df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.progress_apply(lambda x: points(x.new_LV_local_x, x.new_LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))
    
    ## Nearest_point 추출
    df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))
    
    ## Shortest Distance 구하기 : ACT 용
    df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.progress_apply(lambda x: shortest_distance(
        x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
        x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
        x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))
    
    df['potential_conflict_type'], df['degX'] = zip(*df.progress_apply(lambda x: potential_conflict_type(x.LV_type, x.D_gap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.new_local_x, x.new_local_y, x.new_LV_local_x, x.new_LV_local_y, x.V_len, x.LV_len), axis = 1))
    
    print(f'FILE {i} :: Time based SSM')
    # 2차원 SSM
    df['T2'] = df.progress_apply(lambda x: T2(
        x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
        x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
        x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
        x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)
    
    df['TTC'] = df.progress_apply(lambda x: TTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MTTC'] = df.progress_apply(lambda x: MTTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['ACT'] = df.progress_apply(lambda x: ACT(
        x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
        x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
        x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
        x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1)
    
    df['pPET'], df['time_gap'] = zip(*df.progress_apply(lambda x: pPET(
    x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1))
    
#     df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
#     df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)
    
#     df['SSCR_TTC_prime'] = df.progress_apply(lambda x: SSCR_TTC_prime(
#         x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
#         x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
#         x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
#         x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)
    
#     df['SSCR_R'] = df.progress_apply(lambda x: SSCR_R(x.TTC), axis = 1)
#     df['SSCR_Y'] = df.progress_apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
#     df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

    # df = TA(df) # Time to Accident 계산
    # df['critical_speed'] = df.progress_apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
    # df['TA_CS'] = df.progress_apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산
    
    print(f'FILE {i} :: Distance based SSM')
    
    df['PSD'] = df.progress_apply(lambda x: PSD(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, f, g, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['DSS'] = df.progress_apply(lambda x: DSS(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, g, f, delta_t, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    #df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)

    df['PICUD'] = df.progress_apply(lambda x: PICUD(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MTC'] = df.progress_apply(lambda x: MTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.D_x, x.V_len, x.LV_len, x.velocity_x, x.LV_velocity, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    
    df['MMTC'] = df.progress_apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)


    print(f'FILE {i} :: Deceleration based SSM')
    
    df['DRAC'] = df.progress_apply(lambda x: DRAC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.D_x, x.velocity_x, x.LV_velocity_x, x.LV_len, x.V_len, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    #df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

    df['MDRAC'] = df.progress_apply(lambda x: MDRAC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.TTC, delta_t, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['DCIA'] = df.progress_apply(lambda x: DCIA(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, delta_t, x.D_x, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    df['unsafety'] = df.progress_apply(lambda x: unsafety(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, deceleration_rate, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

    #df['RCRI'] = df.progress_apply(lambda x: RCRI(x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.D_x, delta_t, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
    #df['TERCRI'] = df.progress_apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)
    
    
#     df['CIF'] = df.progress_apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
#     df['CIF2'] = df.progress_apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
#     df['ACIF'] = df.progress_apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)

    print('INTEGRATED SSMs')
    df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
    df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
    df['TGIT'] = df.progress_apply(lambda x: TIT(x.time_gap, threshold_TTC, tmsec), axis = 1)
    df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

    df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
    df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
    df['TIMDRAC'] = df.progress_apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)
    
    
    #### SAVE ####
    folder_name = '03_SSM'
    save_name = file_list[i-1]
    save_path = os.path.join(working_dir, folder_name, save_name)
    
    del_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4',
                'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
               'V_LV_point_num', 'V_LV_point_num2',
                'V_LV_point_n', 'V_LV_point_n2',
               'LV_point_n', 'LV_point_n2', 
                'V_LV_point_delta', 'V_LV_shortest_vector']
    
    df.drop(del_cols, axis = 1, inplace = True)
    df.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 
    
    print(f' ###################### FILE {i} IS COMPLETE! ######################')

In [ ]:
# folder_name = '03_SSM'

# for file in tqdm(file_list):
#     file_path = os.path.join(working_dir, folder_name, file)
#     df = pq.read_pandas(file_path).to_pandas()
    
#     df.loc[df['potential_conflict_type'] == 'side_swipe', : 'unsafety'] = None
#     df.loc[df['potential_conflict_type'] == 'angled', : 'unsafety'] = None
    
#     save_path = os.path.join(working_dir, folder_name, file)
#     df.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

In [ ]:
df[df['Lane_record'] == 'U04_U05_U06_U08']['potential_conflict_type']

In [ ]:
# tqdm.pandas()

# folder_name = '03_SSM'
# file_list = ['LV_long_trajectories-0750am-0805am.parquet', 'LV_long_trajectories-0805am-0820am.parquet', 'LV_long_trajectories-0820am-0835am.parquet']

# for file in file_list[1:]:
#     file_path = os.path.join(working_dir, folder_name, file)

#     df = pq.read_pandas(file_path).to_pandas()

#     df['MMTC'] = df.progress_apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.new_local_x, x.new_LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.new_local_y, x.new_LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

#     folder_name = '03_SSM'
#     save_name = file
#     save_path = os.path.join(working_dir, folder_name, save_name)

#     df.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

In [ ]:
tqdm.pandas()

folder_name = '03_SSM'
file_name = 'LV_long_trajectories-0750am-0805am.parquet'
file_path = os.path.join(working_dir, folder_name, file_name)

df = pq.read_pandas(file_path).to_pandas()

# 꼭지점 4개 만들기
# 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.
df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.progress_apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.progress_apply(lambda x: points(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))

## Nearest_point 추출
df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

## Shortest Distance 구하기 : ACT 용
df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.progress_apply(lambda x: shortest_distance(
    x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

## Nearest_point 추출
df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

debug = []

#df['ip1'], df['ip2'], df['cos_theta'], df['theta'], df['degX'],
df['potential_conflict_type'], df['degX'] = zip(*df.progress_apply(lambda x: potential_conflict_type(x.LV_type, x.D_gap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.local_x, x.local_y, x.LV_local_x, x.LV_local_y, x.V_len, x.LV_len), axis = 1))

In [8]:
df['ACT'], df['total_rel'], df['d_delta'] = zip(*df.progress_apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
    x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
    x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1))

delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
              'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
              'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
              'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
              'V_LV_shortest_distance', 'total_rel', 'd_delta']
              #'V_rel', 'acc_rel', 'angle_rel']

df2 = df.copy()
df2.drop(delete_cols, inplace = True, axis = 1)


folder_name = '03_SSM'
save_name = file_name
save_path = os.path.join(working_dir, folder_name, save_name)

df2.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

100%|██████████████████████████████████████████████████████████████████████| 2725522/2725522 [06:06<00:00, 7443.53it/s]


In [9]:
df2[['TTC', 'T2', 'MTTC', 'ACT']]

,TTC,T2,MTTC,ACT
0,NaN,NaN,NaN,NaN
1,77.636705,96.501728,NaN,NaN
2,NaN,NaN,3.635916,2.569808
3,NaN,NaN,2.960940,12.621974
4,NaN,19.065730,2.822975,4.838296
...,...,...,...,...
2725517,NaN,NaN,NaN,13.164829
2725518,NaN,NaN,NaN,NaN
2725519,NaN,NaN,NaN,NaN
2725520,NaN,NaN,NaN,NaN


In [ ]:
# df['pPET'], df['time_gap'] = zip(*df.progress_apply(lambda x: pPET(
# x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
# x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
# x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
# x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1))

# df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
# df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
# df['TGIT'] = df.progress_apply(lambda x: TIT(x.time_gap, threshold_TTC, tmsec), axis = 1)
# df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

# df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
# df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
# df['TIMDRAC'] = df.progress_apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)

In [ ]:
# delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
#               'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
#               'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
#               'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
#               'V_LV_shortest_distance']

# df2 = df.copy()
# df2.drop(delete_cols, inplace = True, axis = 1)

In [ ]:
# folder_name = '03_SSM'
# save_name = file_name
# save_path = os.path.join(working_dir, folder_name, save_name)

# df2.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 